# Quantum Subspace Expansion
Using the ideas from [arXiv:1603.05681](https://arxiv.org/abs/1603.05681), we initially obtain the ground state with VQE and then use that to find the excited states.

In [12]:
import numpy as np
from scipy.linalg import eigh
from openfermion.utils import hermitian_conjugated as hc

from tangelo import SecondQuantizedMolecule
from tangelo.algorithms import FCISolver, CCSDSolver, VQESolver
from tangelo.toolboxes.operators import FermionOperator
from tangelo.toolboxes.qubit_mappings.mapping_transform import fermion_to_qubit_mapping as f2q_mapping


In [13]:
algorithm_resources = {}

# Set Hamiltonian parameters for LiH simulation in active space.
# diatomic_bond_length = 1.5949
diatomic_bond_length = 1.547
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., diatomic_bond_length))]
basis = 'sto-3g'
multiplicity = 1
active_space_start = 1
active_space_stop = 3

mol_LiH = SecondQuantizedMolecule(geometry)# , q=0, spin=0, basis="sto-3g")

In [14]:
vqe_solver = VQESolver({
    'molecule': mol_LiH,
    # 'verbose': True,
})
vqe_solver.build()

In [15]:
vqe_solver.simulate()

-7.882536730639694

In [16]:
op_list = list()
for i in range(2):
    for j in range(i+1, 2):
        op_list += [f2q_mapping(FermionOperator(((2*i, 1), (2*j, 0))), "jw")] #spin-up transition
        op_list += [f2q_mapping(FermionOperator(((2*i+1, 1), (2*j+1, 0))), "jw")] #spin-down transition
        op_list += [f2q_mapping(FermionOperator(((2*i+1, 1), (2*j, 0))), "jw")] #spin-up to spin-down
        op_list += [f2q_mapping(FermionOperator(((2*i, 1), (2*j+1, 0))), "jw")] #spin-down to spin-up

# Compute F and S matrices.
size_mat = len(op_list)
h = np.zeros((size_mat, size_mat))
s = np.zeros((size_mat, size_mat))
state_circuit = vqe_solver.optimal_circuit
for i, op1 in enumerate(op_list):
    for j, op2 in enumerate(op_list):
        h[i, j] = np.real(vqe_solver.backend.get_expectation_value(hc(op1)*vqe_solver.qubit_hamiltonian*op2, state_circuit))
        s[i, j] = np.real(vqe_solver.backend.get_expectation_value(hc(op1)*op2, state_circuit))

label = "quantum_subspace_expansion"
algorithm_resources[label] = vqe_solver.get_resources()
algorithm_resources[label]["n_post_terms"] = len(op_list)**2*algorithm_resources[label]["qubit_hamiltonian_terms"]

In [17]:
e, v = eigh(h,s)
print(f"Quantum Subspace Expansion energies: \n {e}")

Quantum Subspace Expansion energies: 
 [-7.57547495 -7.57538431 -7.57528157 -7.53684578]


In [18]:
mol_N2 = SecondQuantizedMolecule(
    [('N', (0., 0., 0.)), ('N', (0., 0., 1.09))],
    basis='sto-3g',
    frozen_orbitals=5,
)
print(mol_N2.n_active_mos)

vqe_solver_N2 = VQESolver({
    'molecule': mol_N2,
    # 'backend_options': {
    #     'target': 'cirq',
    #     'n_shots': None,
    #     'noise_model': None,
    # },
    # 'verbose': True,
})
vqe_solver_N2.build()
vqe_solver_N2.get_resources()

5


{'qubit_hamiltonian_terms': 188,
 'circuit_width': 10,
 'circuit_depth': 2123,
 'circuit_2qubit_gates': 1808,
 'circuit_var_gates': 184,
 'vqe_variational_parameters': 27}

In [19]:
vqe_solver_N2.simulate()
# with 5 active orbitals: ground state energy = -107.54691139707859
# took 1m 40.7s to run

-107.5469113973229

In [25]:
mol_Cr = SecondQuantizedMolecule(
    [('Cr', (0., 0., 0.))],
    basis='sto-3g',
    frozen_orbitals=10,
)
print(mol_Cr.n_active_mos)

vqe_solver_Cr = VQESolver({
    'molecule': mol_Cr,
    # 'verbose': True,
})
vqe_solver_Cr.build()
vqe_solver_Cr.get_resources()

8


{'qubit_hamiltonian_terms': 2537,
 'circuit_width': 16,
 'circuit_depth': 7529,
 'circuit_2qubit_gates': 6080,
 'circuit_var_gates': 624,
 'vqe_variational_parameters': 90}

In [26]:
vqe_solver_Cr.simulate()

KeyboardInterrupt: 